##Joins in Pyspark

1.   Joins
2.   Joins with selected columns
3.   Joins with filters
4.   Joins with filters & selected columns






In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=1b475099e704b196e29f4a29a44bead3d1a1b142b257e14760048e52df1e903a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
import pyspark
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
c=pyspark.SparkConf().setAppName("test_app").setMaster("local")
sc=pyspark.SparkContext(conf=c)
spark=SparkSession(sc)

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_excel("//content//drive//MyDrive//KSR_DATASETS//2398498-Emp_Training_Data.xlsx")#, header = True, inferSchema = True)

In [ ]:
df2 = pd.read_excel("//content//drive//MyDrive//KSR_DATASETS//2398497-Training.xlsx")#, header = True, inferSchema = True)

In [ ]:
employee = spark.createDataFrame(df1)
training = spark.createDataFrame(df2)

In [ ]:
type(employee)

pyspark.sql.dataframe.DataFrame

In [ ]:
employee.show(5,False)

+----------+---------+----------------+-------------+------+-------------------+--------------+----------+
|EmployeeID|ManagerID|Title           |MaritalStatus|Gender|HireDate           |Dept          |Job Grade |
+----------+---------+----------------+-------------+------+-------------------+--------------+----------+
|1         |16       |Gustavo Achong  |M            |M     |2013-02-02 00:00:00|Sales         |Admin     |
|2         |6        |Catherine Abel  |S            |M     |2013-08-31 00:00:00|Sales         |Management|
|3         |12       |Kim Abercrombie |M            |M     |2014-06-16 00:00:00|Finance       |Admin     |
|4         |3        |Humberto Acevedo|S            |M     |2014-07-10 00:00:00|Logistics     |Admin     |
|5         |263      |Pilar Ackerman  |M            |M     |2014-07-16 00:00:00|Human Resource|Admin     |
+----------+---------+----------------+-------------+------+-------------------+--------------+----------+
only showing top 5 rows



In [ ]:
training.show(5,False)

+-------------------+-----------+----------------------+-----------+----+---------------------+
|Start Date         |Course Code|Course Name           |Employee_ID|Cost|Supplier             |
+-------------------+-----------+----------------------+-----------+----+---------------------+
|2015-09-18 00:00:00|1          |Communication Workshop|1          |1300|Communication Experts|
|2015-09-18 00:00:00|1          |Communication Workshop|4          |1300|Communication Experts|
|2015-09-18 00:00:00|1          |Communication Workshop|25         |1300|Communication Experts|
|2015-09-18 00:00:00|1          |Communication Workshop|33         |1300|Communication Experts|
|2015-09-18 00:00:00|1          |Communication Workshop|44         |1300|Communication Experts|
+-------------------+-----------+----------------------+-----------+----+---------------------+
only showing top 5 rows



In [ ]:
####in pandas
####emp_train = emp.merge(training,left_on="training_id", right_on="", how ="inner")

In [ ]:
emp_training = employee.join(training, employee['EmployeeID']==training["Employee_ID"], "inner")

In [ ]:
emp_training.show(5, False)

+----------+---------+--------------+-------------+------+-------------------+----------+----------+-------------------+-----------+----------------------+-----------+----+---------------------+
|EmployeeID|ManagerID|Title         |MaritalStatus|Gender|HireDate           |Dept      |Job Grade |Start Date         |Course Code|Course Name           |Employee_ID|Cost|Supplier             |
+----------+---------+--------------+-------------+------+-------------------+----------+----------+-------------------+-----------+----------------------+-----------+----+---------------------+
|26        |108      |Marvin Allen  |M            |M     |2015-07-10 00:00:00|Production|Admin     |2017-01-22 00:00:00|2          |Communication Workshop|26         |1300|Communication Experts|
|26        |108      |Marvin Allen  |M            |M     |2015-07-10 00:00:00|Production|Admin     |2017-12-24 00:00:00|16         |Computer Literacy     |26         |2200|Computer Trainers    |
|29        |14       |Osc

In [ ]:
print(len(employee.columns))#first table columns length     ==8
print(len(training.columns))#second table columns length    ==6
print(len(emp_training.columns))#combined columns length ==> ##8+6= 14

8
6
14


## Joins With Selected Columns

In [ ]:
emp_training_c = employee.join(training, employee["EmployeeID"] == training["Employee_ID"], "inner").select(employee['Title'],employee["MaritalStatus"],training["Course Name"],training["Cost"])

In [ ]:
emp_training_c.show(5, False)

+--------------+-------------+----------------------+----+
|Title         |MaritalStatus|Course Name           |Cost|
+--------------+-------------+----------------------+----+
|Marvin Allen  |M            |Communication Workshop|1300|
|Marvin Allen  |M            |Computer Literacy     |2200|
|Oscar Alpuerto|M            |Computer Literacy     |1800|
|Marvin Allen  |M            |Communication Workshop|1300|
|Marvin Allen  |M            |Computer Literacy     |2200|
+--------------+-------------+----------------------+----+
only showing top 5 rows



In [ ]:
#Joins With Filter

##Joins with Filter

In [ ]:
emp_training_filter = employee.join(training, employee['EmployeeID']==training['Employee_ID'],"inner").filter((col("Gender")=="M") & (col("MaritalStatus")=="M") & (training["Cost"] > 2000))

In [ ]:
emp_training_filter.show(20, False)

+----------+---------+----------------+-------------+------+-------------------+--------------+----------+-------------------+-----------+----------------------+-----------+----+-----------------+
|EmployeeID|ManagerID|Title           |MaritalStatus|Gender|HireDate           |Dept          |Job Grade |Start Date         |Course Code|Course Name           |Employee_ID|Cost|Supplier         |
+----------+---------+----------------+-------------+------+-------------------+--------------+----------+-------------------+-----------+----------------------+-----------+----+-----------------+
|1         |16       |Gustavo Achong  |M            |M     |2013-02-02 00:00:00|Sales         |Admin     |2017-12-24 00:00:00|16         |Computer Literacy     |1          |2200|Computer Trainers|
|1         |16       |Gustavo Achong  |M            |M     |2013-02-02 00:00:00|Sales         |Admin     |2018-12-24 00:00:00|16         |Computer Literacy     |1          |2200|Computer Trainers|
|3         |12 

##Joins with both filter & Joins

In [ ]:
emp_train_fcc = employee.join(training, employee["EmployeeID"] == training["Employee_ID"],"inner") \
                        .select("Title","MaritalStatus","Course Name","Cost") \
                        .filter(col('Gender') == "F")

In [ ]:
emp_train_fcc.show(20,False)

+--------------+-------------+----------------------+----+
|Title         |MaritalStatus|Course Name           |Cost|
+--------------+-------------+----------------------+----+
|Milton Albury |M            |Introduction To Excel |2400|
|Milton Albury |M            |Communication Workshop|1300|
|Milton Albury |M            |Communication Workshop|1300|
|Milton Albury |M            |Introduction To Excel |2400|
|Milton Albury |M            |Communication Workshop|1300|
|Milton Albury |M            |Communication Workshop|1300|
|Milton Albury |M            |Introduction To Excel |2400|
|Milton Albury |M            |Communication Workshop|1300|
|Milton Albury |M            |Communication Workshop|1300|
|Margaret Smith|S            |Business Data Analysis|3300|
|Margaret Smith|S            |Advanced Excel        |2300|
|Margaret Smith|S            |Communication Workshop|1300|
|Margaret Smith|S            |Advanced Excel        |2400|
|Margaret Smith|S            |Conflict Management   |140

#Self Join

In [ ]:
rows = [(1,"Santosh",3),(2,"Kiran",1),(3,"Mahesh",None),(4,"Anand",1),(5,"Vijay",1)]
columns = ["EID","emp_name", 'MID']
data = spark.createDataFrame(rows, columns)

In [ ]:
data.show(10, False)

+---+--------+----+
|EID|emp_name|MID |
+---+--------+----+
|1  |Santosh |3   |
|2  |Kiran   |1   |
|3  |Mahesh  |NULL|
|4  |Anand   |1   |
|5  |Vijay   |1   |
+---+--------+----+



In [ ]:
df = data.alias("a").join(data.alias("b"), col("a.MID") == col("b.EID"), "left")

In [ ]:
df.show(10, False)

+---+--------+----+----+--------+----+
|EID|emp_name|MID |EID |emp_name|MID |
+---+--------+----+----+--------+----+
|3  |Mahesh  |NULL|NULL|NULL    |NULL|
|2  |Kiran   |1   |1   |Santosh |3   |
|4  |Anand   |1   |1   |Santosh |3   |
|5  |Vijay   |1   |1   |Santosh |3   |
|1  |Santosh |3   |3   |Mahesh  |NULL|
+---+--------+----+----+--------+----+



In [ ]:
df.select("a.EID","a.emp_name","b.emp_name").show(10,False)

+---+--------+--------+
|EID|emp_name|emp_name|
+---+--------+--------+
|3  |Mahesh  |NULL    |
|2  |Kiran   |Santosh |
|4  |Anand   |Santosh |
|5  |Vijay   |Santosh |
|1  |Santosh |Mahesh  |
+---+--------+--------+



##Changing the column name for the above emp name table

In [ ]:
df.select("a.emp_name",col("b.emp_name").alias("manager_name")).show(10, False)

+--------+------------+
|emp_name|manager_name|
+--------+------------+
|Mahesh  |NULL        |
|Kiran   |Santosh     |
|Anand   |Santosh     |
|Vijay   |Santosh     |
|Santosh |Mahesh      |
+--------+------------+



In [ ]:
##we have divided into parts in the above code here join the whole line of code
finally_Op = data.alias("a").join(data.alias("b"), col("a.MID") == col("b.EID"), "left").select("a.emp_name",col("b.emp_name").alias("manager_name"))#.show(10, False)

In [ ]:
finally_Op = data.alias("a").join(data.alias("b"), col("a.MID") == col("b.EID"), "left") \
                            .select("a.emp_name",col("b.emp_name").alias("Manager_name"))\
                           #.show(10, False)

In [ ]:
finally_Op.show(10,False)

+--------+------------+
|emp_name|Manager_name|
+--------+------------+
|Mahesh  |NULL        |
|Kiran   |Santosh     |
|Anand   |Santosh     |
|Vijay   |Santosh     |
|Santosh |Mahesh      |
+--------+------------+

